In [9]:
#study data
obs     = 'mms1'
year    = '2017'
month   = '06'
day     = '08'
basedir = '/fpiprd1/fpishare/Conrad/Yuggoth/'
#basedir = 'c:/Yuggoth/'

In [10]:
sys.path.append('c:/Users/Conrad/Documents/GitHub/PAD/src/')
sys.path.append('/home/cschiff/PAD/src/')
import Burst_Munger as munge
import Grapher
import PAD

In [74]:
###############################################################################
###############################################################################
################     Time sampling interpolation    ###########################
###############################################################################
###############################################################################
# The idea here is to take two signals like the following:
#
# epoch   0123456789012345678901234567890123456789012345678901234567890
#                  1         2         3         4         5         6
# sample1       x  x  x  x  x  x  x  x  x  x  x  x  x  x  x  x  x  x  x
# sample2     o     o     o     o     o     o     o     o     o     o
#
# and to estimate the value of the x's at the time that the o's 
# occurred, taking into account that the x's may not be available 
# when the o's need them.  In other words, the o-signal is the 
# subscriber and the x-signal is the provider; the request is one-way.  
#
# Despite the way it looks above, generally the o- and x-signals 
# will have no points in common.  The estimated value for the x-signal 
# will be refered to as the x'-signal (or xp in the code)
#
# The requirements are:
#
# 1) The x- and o-signals will be held in numpy arrays with the 
# corresponding datetime objects in another numpy array.  The epoch 
# arrays are assumed to be the same length as the signals themselves 
# (i.e. from a properly formed CDF).
#
# 2) Find the knots in the x-signal that bracket the o-signal
#     a) return the indices of the pair that bracket
#     b) if a pair doesn't exist, flag the x'-signal as masked
#    
# 3) linearly interpolate between the bracketing values and 
# place the derived value into a numpy array of exactly the same 
# length as the o-signal and assume that the x'-signal shares the 
# same epoch array as the o-signal.
#
# Note that the masked array facility gives a nice way to handle things
# since its length will be consistent with the o-signal even if the 
# mask prevents some of the points being used or plotted
###############################################################################

###############################################################################	
#
# bisect_epochs finds a set of epochs in one signal that bracket a target
# epoch.  The indices for these bracketing points are returned.
#
###############################################################################
def bisect_epochs(target_epoch,epochs):

    #test to make sure that target_epoch falls within the epochs array
    #if not send back identical indices (start or end)
    if (target_epoch - epochs[0]).total_seconds() < 0.0:
        return np.nan, np.nan
    if (target_epoch - epochs[-1]).total_seconds() > 0.0:
        return np.nan, np.nan
    
    #bisect to find bracketing points
    low_index  = 0
    high_index = len(epochs)
    
    old_trial_index = low_index
    trial_index     = high_index
    
    counter = 0
    while( abs(old_trial_index - trial_index) > 1 ):
        old_trial_index = trial_index
        trial_index     = np.int(np.floor((high_index+low_index)/2.0))
        trial_epoch     = epochs[trial_index]
        if (trial_epoch - target_epoch).total_seconds() < 0.0:
            low_index  = trial_index
        else:
            high_index = trial_index
        #print counter, low_index, high_index, trial_index
        counter += 1

    #a test for what may happen for an odd number of points 
    #in the epochs array - leaving the bracketing points equal
    if(high_index == low_index):
        #find out if the common epoch is above or below the target
        if (epochs[high_index] - target_epoch).total_seconds() < 0.0:
            high_index = high_index + 1
        else:
            low_index = low_index - 1

    #a test for what may happen when the bracketing points differing by 2
    #this happens when (high_index+low_index) is a multiple of 2
    if(high_index - low_index > 1):
        temp_index = high_index - 1
        if (epochs[temp_index]-target_epoch).total_seconds() < 0.0:
            #bring low up
            low_index = low_index + 1
        else:
            high_index = high_index - 1
            
    return low_index, high_index
    
###############################################################################	
#
# estimate_signal_to_sample returns the estimated value of the x-signal at 
# a specifed o-signal knot
#
###############################################################################    
def estimate_signal_to_sample(o_epoch,x_epochs,x_values):
    
    #find the bracketing point
    low_index, high_index = bisect_epochs(o_epoch,x_epochs)

    #if the bracketing doesn't exist then return a nan
    if np.isnan(low_index) and np.isnan(high_index):
        return np.nan
    
    #if brackets exist, perform linear interpolation
    if high_index-low_index > 1:
        print '''Bracketing problem detected!!! info as follows:
                 o_epoch:    %s
                 low_index:  %s
                 high_index: %s''' % (o_epoch, low_index, high_index)
    delta_t = (x_epochs[high_index] - x_epochs[low_index]).total_seconds()
    delta_x = x_values[high_index]  - x_values[low_index]
    alpha   = (o_epoch - x_epochs[low_index]).total_seconds()
    xp_val  = x_values[low_index] + alpha*delta_x/delta_t
    
    return xp_val    

###############################################################################	
#
# estimate_signal_to_signal returns the estimated value of the x-signal at 
# the o-signal knots
#
###############################################################################    
def estimate_signal_to_signal(o_epochs,x_epochs,x_values):
    num_o   = len(o_epochs)
    est_sig = np.zeros(num_o)
    for i in range(num_o):
        est_sig[i] = estimate_signal_to_sample(o_epochs[i],x_epochs,x_values)
        
    return np.ma.masked_invalid(est_sig)

In [12]:
epoch_strings = ['20170608133933',
                 '20170608134133',
                 '20170608134403',
                 '20170608134623',
                 '20170608134853',
                 '20170608135003',
                 '20170608135133',
                 '20170608135353',
                 '20170608135623',
                 '20170608135803',
                 '20170608135943',
                 '20170608140143',
                 '20170608140353',
                 '20170608140603',
                 '20170608140813',
                 '20170608141033',
                 '20170608141243',
                 '20170608141453',
                 '20170608141703',
                 '20170608141923',
                 '20170608142133',
                 '20170608142303',
                 '20170608143333',
                 '20170608143453']

In [13]:
munge.config_directories(basedir,obs,year,month,day)

In [14]:
Be   = munge.munge_moms(epoch_strings,'des')

* * * * * * * * * * * * * * * * * * * * * * * *


In [15]:
Bi   = munge.munge_moms(epoch_strings,'dis')

* * * * * * * * * * * * * * * * * * * * * * * *


In [16]:
Bfgm = munge.munge_fgm(epoch_strings)

* * * * * * * * * * * * * * * * * * * * * * * *


In [17]:
des_ergs = Be[0]['ergs'][0,:]
dis_ergs = Bi[0]['ergs'][0,:]

In [18]:
bpsd_f = munge.bpsd_dir+'mms1_dsp_fast_l2_bpsd_20170608_v2.2.3.cdf'
epsd_f = munge.epsd_dir+'mms1_dsp_fast_l2_epsd_20170608_v0.6.3.cdf'

In [19]:
munge.bpsd_dir

'/fpiprd1/fpishare/Conrad/Yuggoth/mms1/dsp/fast/l2/bpsd/2017/06/'

In [20]:
bpsd       = pycdf.CDF(bpsd_f)
epsd       = pycdf.CDF(epsd_f)
bpsd_Epoch = np.asarray(bpsd['Epoch'])
bpsd_freq  = np.asarray(bpsd['mms1_b_freq'][:])
epsd_Epoch = np.asarray(epsd['Epoch'])
epsd_freq  = np.asarray(epsd['mms1_e_freq'][:])
bpsd_omni  = np.ma.masked_invalid(np.log10(np.asarray(bpsd['mms1_dsp_bpsd_omni_fast_l2']))).T
epsd_omni  = np.ma.masked_invalid(np.log10(np.asarray(epsd['mms1_dsp_epsd_omni']))).T

/fpiprd1/anaconda/fpi/anaconda2_4.4.0/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in log10
/fpiprd1/anaconda/fpi/anaconda2_4.4.0/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: divide by zero encountered in log10


In [21]:
bpsd.close()
epsd.close()

In [77]:
for i in range(len(Be)):
    Be[i]['derived_fgm_mag'] = estimate_signal_to_signal(Be[i]['epochs'],\
                                                         Bfgm[i]['epochs'],\
                                                         Bfgm[i]['Bgsm'][:,3])  
    Bi[i]['derived_fgm_mag'] = estimate_signal_to_signal(Bi[i]['epochs'],\
                                                         Bfgm[i]['epochs'],\
                                                         Bfgm[i]['Bgsm'][:,3])  
    

In [85]:
mu_0 = 4*np.pi*1e-7
for i in range(len(Be)):
    Be[i]['beta'] = Be[i]['trace_p']*2.0*mu_0/\
                    (Be[i]['derived_fgm_mag']*Be[i]['derived_fgm_mag']*1.0e-9)
    Bi[i]['beta'] = Bi[i]['trace_p']*2.0*mu_0/\
                    (Bi[i]['derived_fgm_mag']*Bi[i]['derived_fgm_mag']*1.0e-9)

In [88]:
print Be[0]['derived_fgm_mag'][18]
print Be[0]['trace_p'][18]
print Be[0]['trace_p'][18]*2.0*mu_0/(Be[0]['derived_fgm_mag'][18]**2*1.0e-9)
print Be[0]['beta'][18]

11.5441829162
0.00781825
0.147442628706
0.147442626596


In [104]:
starttime = dt.datetime(2017,6,8,13,45,0)
deltat = dt.timedelta(seconds=300)
stoptime  = starttime + deltat
#starttime = Bi[0]['epochs'][0]
#deltat = dt.timedelta(seconds=120)
#stoptime  = starttime + deltat
#stoptime  = Bi[-1]['epochs'][-1]

In [ ]:
N = 9
fig  = plt.figure(figsize=(16,N*4))
ax1  = fig.add_subplot(N,1,1)               #mag field
ax2  = fig.add_subplot(N,1,2,sharex=ax1)    #ion bulk v
ax3  = fig.add_subplot(N,1,3,sharex=ax1)    #dis omni E-t spectrogram
ax3a = fig.add_subplot(N,1,4,sharex=ax1)    #des omni E-t spectrogram
ax4  = fig.add_subplot(N,1,5,sharex=ax1)    #bfield psd
ax5  = fig.add_subplot(N,1,6,sharex=ax1)    #efield psd
ax6  = fig.add_subplot(N,1,7,sharex=ax1)    #perp and par temperatures
ax7  = fig.add_subplot(N,1,8,sharex=ax1)    #density
ax8  = fig.add_subplot(N,1,9,sharex=ax1)    #plasma beta
flag = 0
imoms_min = 2
imoms_max = 6
emoms_min = 4
emoms_max = 8
for d in range(len(Be)):
    if flag == 0:
        ax1.plot(Bfgm[d]['epochs'],Bfgm[d]['Bgsm'][:,0],'b-',label='Bx')
        ax1.plot(Bfgm[d]['epochs'],Bfgm[d]['Bgsm'][:,1],'g-',label='By')
        ax1.plot(Bfgm[d]['epochs'],Bfgm[d]['Bgsm'][:,2],'r-',label='Bz')
        ax1.plot(Bfgm[d]['epochs'],Bfgm[d]['Bgsm'][:,3],'k-',label='|B|')        
        ax2.plot(Bi[d]['epochs'],Bi[d]['bulk_vs'][:,0],'b-',label='Vx')
        ax2.plot(Bi[d]['epochs'],Bi[d]['bulk_vs'][:,1],'g-',label='Vy')
        ax2.plot(Bi[d]['epochs'],Bi[d]['bulk_vs'][:,2],'r-',label='Vz')
        ax6.plot(Bi[d]['epochs'],Bi[d]['T_par'],'k-',label='Ti_par')
        ax6.plot(Bi[d]['epochs'],Bi[d]['T_perp'],'r-',label='Ti_perp')        
        ax6.plot(Be[d]['epochs'],Be[d]['T_par'],'b-',label='Te_par')
        ax6.plot(Be[d]['epochs'],Be[d]['T_perp'],'g-',label='Te_perp')  
        ax7.plot(Bi[d]['epochs'],Bi[d]['num_den'],'r-',label='n_i')
        ax7.plot(Be[d]['epochs'],Be[d]['num_den'],'k-',label='n_e')        
        ax8.plot(Bi[d]['epochs'],Bi[d]['beta'],'r-',label='beta_i')
        ax8.plot(Be[d]['epochs'],Be[d]['beta'],'k-',label='beta_e')        

        flag = 1
    else:
        ax1.plot(Bfgm[d]['epochs'],Bfgm[d]['Bgsm'][:,0],'b-')
        ax1.plot(Bfgm[d]['epochs'],Bfgm[d]['Bgsm'][:,1],'g-')
        ax1.plot(Bfgm[d]['epochs'],Bfgm[d]['Bgsm'][:,2],'r-')
        ax1.plot(Bfgm[d]['epochs'],Bfgm[d]['Bgsm'][:,3],'k-')        
        ax2.plot(Bi[d]['epochs'],Bi[d]['bulk_vs'][:,0],'b-')
        ax2.plot(Bi[d]['epochs'],Bi[d]['bulk_vs'][:,1],'g-')
        ax2.plot(Bi[d]['epochs'],Bi[d]['bulk_vs'][:,2],'r-')
        ax6.plot(Bi[d]['epochs'],Bi[d]['T_par'],'k-')
        ax6.plot(Bi[d]['epochs'],Bi[d]['T_perp'],'r-')                
        ax6.plot(Be[d]['epochs'],Be[d]['T_par'],'b-')
        ax6.plot(Be[d]['epochs'],Be[d]['T_perp'],'g-')
        ax7.plot(Bi[d]['epochs'],Bi[d]['num_den'],'r-')
        ax7.plot(Be[d]['epochs'],Be[d]['num_den'],'k-')        
        ax8.plot(Bi[d]['epochs'],Bi[d]['beta'],'r-')
        ax8.plot(Be[d]['epochs'],Be[d]['beta'],'k-')        
    cbpatch3 = ax3.pcolormesh(Bi[d]['epochs'],\
                              dis_ergs,\
                              np.ma.masked_invalid(np.log10(Bi[d]['omnis'].T)),\
                              cmap=cmap.jet,\
                              shading='gouraud',
                              vmin = imoms_min,
                              vmax = imoms_max)

    cbpatch3a = ax3a.pcolormesh(Be[d]['epochs'],\
                              des_ergs,\
                              np.ma.masked_invalid(np.log10(Be[d]['omnis'].T)),\
                              cmap=cmap.jet,\
                              shading='gouraud',
                              vmin = emoms_min,
                              vmax = emoms_max)
    
#xmin,xmax = ax1.get_xlim()   

cbpatch4 = ax4.pcolormesh(bpsd_Epoch,bpsd_freq,bpsd_omni,cmap=cmap.jet)
cbpatch5 = ax5.pcolormesh(epsd_Epoch,epsd_freq,epsd_omni,cmap=cmap.jet)

cb3ax  = fig.add_axes(Grapher.cbar_position(ax3,0.01,0.01))
cb3aax = fig.add_axes(Grapher.cbar_position(ax3a,0.01,0.01))
cb4ax  = fig.add_axes(Grapher.cbar_position(ax4,0.01,0.01))
cb5ax  = fig.add_axes(Grapher.cbar_position(ax5,0.01,0.01))

fig.colorbar(cbpatch3,cax=cb3ax,ticks=np.array(range(imoms_min,imoms_max+1)),format=ticker.FormatStrFormatter('$10^{%d}$'))
fig.colorbar(cbpatch3a,cax=cb3aax,ticks=np.array(range(emoms_min,emoms_max+1)),format=ticker.FormatStrFormatter('$10^{%d}$'))
fig.colorbar(cbpatch4,cax=cb4ax,format=ticker.FormatStrFormatter('$10^{%d}$'))
fig.colorbar(cbpatch5,cax=cb5ax,format=ticker.FormatStrFormatter('$10^{%d}$'))

ax1.set_ylabel('Magnetic Field (nT)\n GSM FGM',fontsize=16)
ax1.legend(fontsize=18)    

ax2.set_ylabel('Ion Bulk Velocity (km/s)\n GSE',fontsize=16)
ax2.legend(fontsize=18)    

ax3.set_yscale('log')        
ax3.set_ylabel('DIS Energy (eV)',fontsize=16)
cb3ax.set_ylabel('keV/(cm^2 s sr keV)',fontsize=16)

ax3a.set_yscale('log')        
ax3a.set_ylabel('DES Energy (eV)',fontsize=16)
cb3aax.set_ylabel('keV/(cm^2 s sr keV)',fontsize=16)

ax4.set_yscale('log')
ax4.set_ylim([bpsd_freq[0],bpsd_freq[-1]])
ax4.set_xlabel('Epoch',fontsize=16)
ax4.set_ylabel('Frequency (Hz)',fontsize=16)
cb4ax.set_ylabel('(nT)^2/Hz',fontsize=16)
#ax4.set_xlim([xmin,xmax])

ax5.set_yscale('log')
ax5.set_ylim([epsd_freq[0],epsd_freq[-1]])
ax5.set_ylabel('Frequency (Hz)',fontsize=16)
cb5ax.set_ylabel('(V/m)^2/Hz',fontsize=16)

ax6.set_yscale('log')
ax6.set_ylabel('Temperature (eV)',fontsize=16)
ax6.legend(fontsize=18)    

ax7.set_yscale('linear')
ax7.set_ylabel('Number Density',fontsize=16)
ax7.legend(fontsize=18)    

ax8.set_yscale('log')
ax8.set_ylabel('plasma beta',fontsize=16)
ax8.legend(fontsize=18)    
ax8.set_xlabel('Epoch',fontsize=16)

ax1.set_xlim([mpl.dates.date2num(starttime),mpl.dates.date2num(stoptime)])

plt.show()

/fpiprd1/anaconda/fpi/anaconda2_4.4.0/lib/python2.7/site-packages/ipykernel/__main__.py:52: RuntimeWarning: divide by zero encountered in log10
/fpiprd1/anaconda/fpi/anaconda2_4.4.0/lib/python2.7/site-packages/ipykernel/__main__.py:56: RuntimeWarning: divide by zero encountered in log10


In [103]:
fig.savefig('/fpishare/Conrad/Science/June_08_2017_leg1.png')